# Linear Programming
***
## Maximum Return Of Portfolio
> <font size="4">μ<sub>p</sub> = μ<sup>T</sup>w</font>

## Constrained Optimization
> <font size="4">max<sub>w</sub> μ<sup>T</sup>W, </font>

> <font size="4">where sum of matrix w must be 1, Hence, </font>

> <font size="4">1<sub>D</sub><sup>T</sup>w = 1</font>

## Realistic Constraints

> <font size="4">w<sub>i</sub> ≥ 0 (value of weight i must be greater than or equal to 0)</font>

> <font size="4">w<sub>i</sub> ≤ .5 (value of weight i must be smaller than or equal to 0.5)</font>

## Minimum Return Of Portfolio
> <font size="4">min<sub>w</sub> μ<sup>T</sup>W, </font>

## Optimization can be done using <code>scipy.optimize.linprog</code>

scipy.optimize.linprog(
    <br>
    c, 
    <br>
    A_ub=None, 
    <br>
    B_ub=None, 
    <br>
    A_eq=None, 
    <br>
    B_eq=None, 
    <br>
    bounds=None)
    
min<sub>c</sub> μ<sup>T</sup>W,


such that </font>

A<sub>ub</sub>x ≤ b<sub>ub,</sub>
<br>
A<sub>eq</sub>x ≤ b<sub>eq,</sub>
<br>
l ≤ x ≤ u

In [3]:
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
import numpy as np

In [4]:
names = ['GOOG', 'SBUX', 'KSS', 'NEM']

#Get the stock quote
returns = pd.DataFrame()
for n in names:
    df = pdr.DataReader(n, data_source='yahoo', start='2019-01-01', end='2022-01-01')
    ret = df['Close'].pct_change(1).dropna()
    returns.loc[:, n + '_ret'] = ret

returns

,GOOG_ret,SBUX_ret,KSS_ret,NEM_ret
Date,,,,
2019-01-02,0.009888,-0.001242,0.022008,-0.004906
2019-01-03,-0.028484,-0.043377,-0.021091,-0.002900
2019-01-04,0.053786,0.033155,0.000603,0.000000
2019-01-07,-0.002167,0.000000,0.024093,-0.011926
2019-01-08,0.007385,0.000787,0.017203,0.002944
...,...,...,...,...
2021-12-27,0.006263,0.016463,0.012154,0.010919
2021-12-28,-0.010914,0.011819,-0.031496,0.001828
2021-12-29,0.000386,0.007009,0.017886,0.005474


In [6]:
# Mean return of each stock, also the coefficients of the linear objective function to be minimized.
mean_return = returns.mean()

# D = Number of asset in porfolio
D = len(mean_return) 
D

4

In [7]:
# The equality constraint matrix. 
# Each row of A_eq specifies the coefficients of a linear equality constraint on x.
A_eq = np.ones((1, D)) 

# The equality constraint vector. Each element of A_eq @ x must equal the corresponding element of b_eq.
b_eq = np.ones(1)     

### Note: The bounds are by default (0, None) unless otherwise specified.
# bounds = None
# A sequence of (min, max) pairs for each element in x, defining the minimum and maximum values of that decision variable.
bounds = [(-0.5, None)] * D
bounds 

[(-0.5, None), (-0.5, None), (-0.5, None), (-0.5, None)]

In [12]:
# minimize
res = linprog(
    c=mean_return,  
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds
)
res

     con: array([-1.195674e-09])
     fun: -0.00046549748155553193
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([-0.49999946, -0.49999999,  2.49999939, -0.49999994])

The status is 0 or the success is True indicates that the optimization was successful.

In [13]:
# The current value of the objective function c @ x.
min_return = res.fun

In [14]:
# Maximize
res = linprog(
    c=-mean_return, 
    A_eq=A_eq,  
    b_eq=b_eq, 
    bounds=bounds
)
res

     con: array([-1.02755093e-09])
     fun: -0.0025781822814766567
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 2.49999946, -0.49999995, -0.49999951, -0.5       ])

In [15]:
max_return = -res.fun

In [22]:
print(f"Minimum Return: {min_return}\nMaximum Return: {max_return}")

Minimum Return: -0.00046549748155553193
Maximum Return: 0.0025781822814766567
